# Portfolio Project: Safety Option

**Names:** Deniz Makul, Richard Sha, Mitchel Shen

**Due:** 11/25/2022 at 11:59pm ET on Dropbox

**What to submit:** your .ipynb (python code in jupyter notebooks) file as well as a pdf with all the output from your code.  




In [2]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
#Getting tickers from csv file
csv_tickers = pd.read_csv('Tickers_Example.csv', header = None)

In [4]:
#Drops the tickers with .TO ending from the csv_tickers dataframe
mask = csv_tickers.loc[csv_tickers[0].str.contains('.TO')]
csv_tickers.drop(index = mask.index, inplace = True)
csv_tickers.reset_index(drop = True, inplace = True)
csv_tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


In [15]:
volume_start_date = '2022-01-01'
volume_end_date = '2022-10-31'

info = yf.Ticker('AAPL')
hist = info.history(start = volume_start_date, end = volume_end_date)

hist.filter(like='2022-02')

""
Date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
...
2022-10-24
2022-10-25
2022-10-26


In [20]:
volume_start_date = '2022-01-01'
volume_end_date = '2022-10-31'

months_to_drop = []

i = 1
while i<11: 
    info = yf.Ticker('AAPL')
    hist = info.history(start = f'2022-{i}-01', end = f'2022-{i+1}-01')
    hist = hist.dropna()
    if (len(hist) < 20):
        months_to_drop.append(i)
    i+=1
    
print(months_to_drop)

[2]


In [5]:
#Drops the tickers that are delisted or have a average monthly volume from 
# January 1st to October 31st of less that 200,000
volume_index_drop = []

i = 0
while i < len(csv_tickers):
    stock_data = yf.Ticker(csv_tickers[0][i])
    stock_hist = stock_data.history(start = volume_start_date, end = volume_end_date, interval = '1mo')
    stock_volume = stock_hist['Volume']
    
    stock_volume = stock_volume.dropna()
    stock_volume.drop(index = months_to_drop, inplace = True)
    #print(stock_volume)
    mean = (stock_volume.mean())
    
    if(np.isnan(mean) or mean < 200000):
        volume_index_drop.append(i)
    elif(stock_data.info['market'] != 'us_market'):
        volume_index_drop.append(i)
    i += 1

Date
2022-01-01    2.108446e+09
2022-02-01    1.627516e+09
2022-02-04             NaN
2022-03-01    2.180800e+09
2022-04-01    1.687796e+09
2022-05-01    2.401040e+09
2022-05-06             NaN
2022-06-01    1.749100e+09
2022-07-01    1.447125e+09
2022-08-01    1.510240e+09
2022-08-05             NaN
2022-09-01    2.084723e+09
2022-10-01    1.868140e+09
Name: Volume, dtype: float64
Date
2022-01-01    168562800.0
2022-01-13            NaN
2022-02-01    165818900.0
2022-03-01    151159500.0
2022-04-01    178925400.0
2022-04-13            NaN
2022-05-01    143523500.0
2022-06-01    132591100.0
2022-07-01    116516600.0
2022-07-14            NaN
2022-08-01    109508700.0
2022-09-01    125557500.0
2022-10-01    114545700.0
2022-10-13            NaN
Name: Volume, dtype: float64
Date
2022-01-01    155539600.0
2022-01-13            NaN
2022-02-01    111246800.0
2022-03-01    123453100.0
2022-04-01    102385000.0
2022-04-13            NaN
2022-05-01    125858700.0
2022-06-01    112185100.0
2022

Date
2022-01-01    470897500.0
2022-02-01    384992400.0
2022-03-01    403087300.0
2022-04-01    363967100.0
2022-05-01    397427200.0
2022-06-01    323802700.0
2022-07-01    292472200.0
2022-08-01    317943000.0
2022-08-30            NaN
2022-09-01    332839800.0
2022-10-01    317684500.0
Name: Volume, dtype: float64
Date
2022-01-01    615510000.0
2022-02-01    732650000.0
2022-03-01    637420000.0
2022-04-01    564940000.0
2022-05-01    717486000.0
2022-06-01    627132000.0
2022-07-01    638766600.0
2022-07-18            NaN
2022-08-01    430961500.0
2022-09-01    532993100.0
2022-10-01    584537500.0
Name: Volume, dtype: float64
Date
2022-01-01    373950200.0
2022-01-05            NaN
2022-02-01    288730100.0
2022-03-01    390796800.0
2022-04-01    303650400.0
2022-04-05            NaN
2022-05-01    313630400.0
2022-06-01    268201900.0
2022-07-01    239623100.0
2022-07-05            NaN
2022-08-01    258430300.0
2022-09-01    248558100.0
2022-10-01    318197200.0
2022-10-05       

Date
2022-01-01    340225400.0
2022-02-01    288724300.0
2022-02-08            NaN
2022-03-01    470064200.0
2022-04-01    245559100.0
2022-05-01    273965500.0
2022-06-01    344160900.0
2022-07-01    247148900.0
2022-08-01    242709600.0
2022-09-01    246233600.0
2022-10-01    354101300.0
Name: Volume, dtype: float64
Date
2022-01-01    111858200.0
2022-02-01    132953900.0
2022-02-18            NaN
2022-03-01    126815500.0
2022-04-01    115727900.0
2022-05-01    112507500.0
2022-06-01    105980700.0
2022-07-01     87323500.0
2022-08-01     82912000.0
2022-09-01     82457500.0
2022-10-01     96021400.0
Name: Volume, dtype: float64
Date
2022-01-01    41589800
2022-02-01    44388100
2022-03-01    52220200
2022-04-01    36943600
2022-05-01    50701700
2022-06-01    52007200
2022-07-01    33021400
2022-08-01    37720400
2022-09-01    47428800
2022-10-01    42524300
Name: Volume, dtype: int64
Date
2022-01-01    1.828522e+09
2022-01-07             NaN
2022-02-01    1.300036e+09
2022-03-01  

In [6]:
#Drops invalid tickers
csv_tickers.drop(index = volume_index_drop, inplace = True)
csv_tickers.reset_index(drop = True, inplace = True)
csv_tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AIG


In [ ]:
# Richard

In [ ]:
# Deniz

In [ ]:
# Mitchel